In [12]:
import numpy as np
import torch as th
from torchvision import datasets
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import BatchSampler, RandomSampler, SequentialSampler
import syft as sy
from syft.frameworks.torch.fl.loss_fn import nll_loss
from functools import reduce
import time
import sys 

hook = sy.TorchHook(th)

device = th.device("cpu")
worker_id = "alice"
batch_size = 64
lr = 0.1 
epoch_num = 20

In [13]:
p_num = 8850

class Net(th.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, p_num)
        self.fc2 = nn.Linear(p_num, 500)
        self.fc3 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
#         x = self.fc2(x)
        
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=1)

model = Net()

In [26]:
total_num_params = 0
for params in list(model.parameters()):
    params_shape = list(params.size())
    num_params = reduce(lambda x, y: x*y, params_shape)
    total_num_params += num_params

In [27]:
total_num_params

11544930

In [28]:
11544930 * 4

46179720